##### Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tslearn
from tslearn.clustering import TimeSeriesKMeans
from sklearn import cluster
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances
from gap_statistic import OptimalK
import matplotlib.pyplot as plt
import seaborn as sns

KeyboardInterrupt: 

##### Import data

In [ ]:
data_trimmed = pd.read_csv('Data/data_trimmed.csv') #return data
sp500 = pd.read_csv('Data/sp500.csv') #sp500 return data
ticker_data = pd.read_csv('Data/Tickers.csv') #fundno-ticker
nameticker = pd.read_excel('Data/nameticker.xlsx',sheet_name = 'Category') #ticker-name-morningstar category
holding_asset = pd.read_csv('Data/Summary_Updated.csv') #holding_asset data
holding_asset = holding_asset.iloc[:, [0, 2]+[i for i in range(17, 30)]]
fund_mornstar = pd.read_csv('Data\Summary_Updated.csv')

<li>Data wrangling

In [ ]:
rec = {}
ticker_fundno = {}
fundno_ticker = {}

for i in range(nameticker.shape[0]):
    rec[nameticker.Name[i]] = nameticker.Ticker[i]

for i in range(ticker_data.shape[0]):
    if pd.isnull(ticker_data.ticker[i]):
        continue
    ticker_fundno[ticker_data.ticker[i]] = ticker_data.crsp_fundno[i]
    fundno_ticker[ticker_data.crsp_fundno[i]] = ticker_data.ticker[i]

In [ ]:
sp500.date = pd.to_datetime(sp500.date)

In [ ]:
data_trimmed = data_trimmed.set_index('date')
data_trimmed.index = pd.to_datetime(data_trimmed.index)
data_trimmed = data_trimmed.astype(float)

In [ ]:
holding_asset['per_equity'] = holding_asset['per_com'] + holding_asset['per_pref'] + holding_asset['per_eq_oth']
holding_asset['per_bond'] = holding_asset['per_conv'] + holding_asset['per_corp'] + holding_asset['per_muni'] + holding_asset['per_govt']
holding_asset['per_sec'] = holding_asset['per_abs'] + holding_asset['per_mbs'] + holding_asset['per_fi_oth'] + holding_asset['per_oth']
holding_asset.caldt = pd.to_datetime(holding_asset.caldt, format='%Y%m%d')
holding_asset = holding_asset[['crsp_fundno', 'caldt', 'per_cash', 'per_equity', 'per_bond', 'per_sec']]
holding_asset.columns = ['crsp_fundno', 'caldt', 'cash', 'equity', 'bond', 'security']

In [ ]:
fund_mornstar = fund_mornstar[['crsp_fundno', 'caldt', 'lipper_class_name']]
fund_mornstar.caldt = pd.to_datetime(fund_mornstar.caldt, format='%Y%m%d')

In [ ]:
list(holding_asset.columns)[2:]

##### Clustering - First layer based on holding asset

In [ ]:
class first_layer_clustering:
    def __init__(self, clustering_year, ret_data, asset_data, mrnstar_class_data):
        self.clustering_year = clustering_year
        self.ret_data = ret_data[ret_data.index.year == clustering_year]
        self.cumret_data = (self.ret_data+1).cumprod()
        self.asset_data = asset_data[(asset_data.caldt.dt.year == clustering_year) & (asset_data.caldt.dt.month == 12)]
        self.asset_type = list(self.asset_data.columns)[2:]
        self.mrnstar_data = mrnstar_class_data[(mrnstar_class_data.caldt.dt.year == (clustering_year)) & (mrnstar_class_data.caldt.dt.month == 12)]
        self.feature = None
        self.feature_nostd = None
        self.label = None
        self.k = None

    def init_features(self):
        
        def standardization(df):
            for column in df.columns:
                df[column] = (df[column] - df[column].mean())/np.std(df[column])
            return df
        
        feature = pd.DataFrame(index=self.ret_data.columns)
        funds = set(self.asset_data.crsp_fundno.values)
        
        for index in feature.index:
            if int(index) in funds:
                for a in self.asset_type:A
                    feature.loc[index, a] = self.asset_data[self.asset_data.crsp_fundno == int(index)][a].values[0]
            else:
                for a in self.asset_type:
                    feature.loc[index, a] = np.NaN
        
        feature.dropna(axis=0, inplace=True)
        self.feature = feature
        self.ret_data = self.ret_data[self.feature.index]
        self.cumret_data = self.cumret_data[self.feature.index]
        
        self.feature_nostd = self.feature.copy()
        self.feature = standardization(self.feature)
        self.feature = np.round(self.feature, 4)
        return self.feature, self.feature_nostd
    
    def asset_based_clustering(self, log=False, identical_asset=3, argsort=False):
        #Functions will be used
        def get_silhouette(feature, log=False):
            from sklearn import metrics
            score = []
            mx = 0
            res = None
            for i in range(2, 30):
                k=i
                clustering = cluster.AgglomerativeClustering(n_clusters=k).fit(feature)
                cluster_label = clustering.labels_
                score.append(metrics.silhouette_score(feature, cluster_label, metric='euclidean'))
                if score[-1] >= mx:
                    mx = score[-1]
                    res = i
            if log:
                plt.plot(list(range(2, 30)), score)
                plt.xlim(2, 30)
                print('The best k determined by statistical approach is', res)
            return res


        def get_new_center(feature, label, k):
            cluster_center_init = []
            for i in range(k):
                df = feature[label == i]
                cur_cluster_center = df.median(axis=0)
                if len(df) == 0:
                    continue
                sit = False
                for ass in list(df.columns):
                    ass_25 = np.percentile(df[ass], 25)
                    ass_75 = np.percentile(df[ass], 75)
                    low = max(ass_25 - 1.5*(ass_75-ass_25), df[ass].min())
                    high = min(ass_75 + 1.5*(ass_75-ass_25), df[ass].max())
                    
                    if ass_75 - ass_25 > 20:
                        cluster_center_init.append(df[df[ass] == np.sort(df[ass])[len(df[ass])*3//4]].iloc[0, :])
                        cluster_center_init.append(df[df[ass] == np.sort(df[ass])[len(df[ass])*1//4]].iloc[0, :])
                        sit = True
                    elif high - low >= 50:
                        cluster_center_init.append(df[df[ass] == np.sort(df[df[ass] >= high][ass])[sum(df[ass] >= high)//2]].iloc[0, :])
                        cluster_center_init.append(df[df[ass] == np.sort(df[df[ass] <= low][ass])[sum(df[ass] <= low)//2]].iloc[0, :])
                        sit = True
                    elif high - ass_75 > 20 or sum(df[ass] >= high) >= 10:
                        cluster_center_init.append(df[df[ass] == np.sort(df[df[ass] >= high][ass])[sum(df[ass] >= high)//2]].iloc[0, :])
                        sit = True
                    elif ass_25 - low > 20 or sum(df[ass] <= low) >= 10:
                        cluster_center_init.append(df[df[ass] == np.sort(df[df[ass] <= low][ass])[sum(df[ass] <= low)//2]].iloc[0, :])
                        sit = True
                    if sit == True:
                        break
                if sit == False:
                    cluster_center_init.append(cur_cluster_center)
            cluster_center_init = np.array(cluster_center_init)
            return np.unique(cluster_center_init, axis=0)

        #Merge clusters with very similar cluster centers
        def merge_cluster(feature, label, k, identical_asset=3, argsort=False):
            clust_centers = []
            for i in range(k):
                similar = False
                df = feature[label == i]
                center = df.median(axis=0)
                for c in clust_centers:
                    if argsort:
                        criteria = (sum((center - c).abs() <= 10) >= identical_asset and max((center - c).abs()) <= 20) or \
                    (all(np.argsort(np.floor(center)) == np.argsort(np.floor(c))) and all((center - c).abs() <= 25))
                    else:
                        criteria = sum((center - c).abs() <= 10) >= identical_asset and max((center - c).abs()) <= 20

                    if criteria:
                        similar = True
                        break
                if not similar:
                    clust_centers.append(center)
            clust_centers = np.array(clust_centers)
            return np.unique(clust_centers, axis=0)

        #Look for cluster groups with less than 10 components and check if they can be merged into other clusters
        def merge_outlier(label, data_nostd, log=False):
            n = len(np.unique(label))
            centers = {}
            for i in range(n):
                centers[i] = data_nostd[label==i].median(axis=0)

            for i in range(n):
                if sum(label == i) <= 10:
                    mn = float('inf')
                    res = None
                    for j in range(n):
                        if j == i:
                            continue
                        if j in centers:
                            distance = (centers[i] - centers[j]).abs().sum()
                        else:
                            continue
                        if all(np.argsort(np.floor(centers[i])) == np.argsort(np.floor(centers[j]))) and distance < mn:
                            mn = distance
                            res = j
                    if res is not None:
                        label = np.where(label == i, res, label)
                        del centers[i]
                        if log:
                            print(f'cluster{i} get merged into cluster{res}')

            new_n = len(np.unique(label))
            numbers = sorted(np.unique(label), reverse=True)
            l = 0
            for i in range(new_n):
                if i not in numbers:
                    label = np.where(label == numbers[l], i, label)
                    l += 1
            return label

        #Plot the boxplot distribution of holding assets of each cluster
        def plot_asset(asset, label, k, asset_type):
            width = 20
            height = ((k//10)+1)*15
            plt.figure(figsize=(width, height))
            for i in range(1, k+1):
                plt.subplot(int(np.ceil(k/4)), 4, i)
                data = [asset.loc[asset.index[label == i-1], :][col] for col in asset_type]
                plt.boxplot(data)
                plt.xlabel(asset_type)
                plt.title(f'cluster_{i-1} with {sum(label==i-1)} components')
            plt.show()


        #Non function part
        k = get_silhouette(self.feature, log)
        h_clustering = cluster.AgglomerativeClustering(n_clusters=k, linkage='ward').fit(self.feature)
        cluster_label = h_clustering.labels_
        print('Best number of clusters for hierarchical clustering is', k)

        #Screen out the outliers
        length1 = -1
        length2 = -2
        print('Starting searching and grouping outliers')
        #Look for the outliers
        while length1 != length2:
            cluster_center_init = get_new_center(self.feature_nostd, cluster_label, k)
            length1 = k = len(cluster_center_init)
            clustering = cluster.KMeans(n_clusters=k, init=cluster_center_init, n_init=1).fit(self.feature_nostd)
            cluster_label = clustering.labels_

            cluster_center_init = get_new_center(self.feature_nostd, cluster_label, k)
            length2 = k = len(cluster_center_init)
            clustering = cluster.KMeans(n_clusters=k, init=cluster_center_init, n_init=1).fit(self.feature_nostd)
            cluster_label = clustering.labels_
            if log:
                print('split outliers:', length1, length2)
        print('Split finished. No more outliers could be found in the cluster according to the given criteria.')


        #Merge cluster centers that are very close
        length1 = -1
        length2 = -2
        while length1 != length2:
            new_cluster_center_init = merge_cluster(self.feature_nostd, cluster_label, k, identical_asset, argsort)
            new_cluster_center_init = new_cluster_center_init[~np.isnan(new_cluster_center_init).any(axis=1)]
            length1 = k = len(new_cluster_center_init)
            clustering = cluster.KMeans(n_clusters=k, init=new_cluster_center_init, n_init=1).fit(self.feature_nostd)
            cluster_label = clustering.labels_

            new_cluster_center_init = merge_cluster(self.feature_nostd, cluster_label, k, identical_asset, argsort)
            new_cluster_center_init = new_cluster_center_init[~np.isnan(new_cluster_center_init).any(axis=1)]
            length2 = k = len(new_cluster_center_init)
            clustering = cluster.KMeans(n_clusters=k, init=new_cluster_center_init, n_init=1).fit(self.feature_nostd)
            cluster_label = clustering.labels_
            if log:
                print('converge centers:', length1, length2)
        print('Merging finished. No more centroids could be merged cross different clusters according to the given criteria.')

        print('Start to merge outlier clusters.')
        cluster_label = merge_outlier(cluster_label, self.feature_nostd, log)
        self.label = cluster_label
        k = len(np.unique(cluster_label))
        self.k = k
        print('The final clusters are', k)

        if log:
            plot_asset(self.feature_nostd, cluster_label, k, self.asset_type)
        return cluster_label
    
    def write_csv(self, fundno_ticker, loc='final_output'):
        def label_cluster(l, data_nostd):
            k = len(np.unique(l))
            labelling = {}
            ref = {0:data_nostd.columns[0], 
                   1:data_nostd.columns[1],
                   2:data_nostd.columns[2],
                   3:data_nostd.columns[3]}
            for i in range(k):
                index = data_nostd[l==i].median(axis=0)
                if index[0] <= -20:
                    if index[1] >= 80:
                        labelling[i] = 'Leveraged ' + ref[1] + ' group'
                    elif index[2] >= 80:
                        labelling[i] = 'Leveraged ' + ref[2] + ' group'
                    elif index[3] >= 80:
                        labelling[i] = 'Leveraged ' + ref[3] + ' group'
                    else:
                        labelling[i] = 'Leveraged mixed group'
                elif max(index) >= 80:
                    labelling[i] = np.argmax(index) + ' driven group'
                elif sum(list(map(lambda x: x >= 10, index))) >= 3:
                    labelling[i] = 'Deversified group'
                elif sum(list(map(lambda x: x >= 25, index))) >= 2:
                    if index[0] >= 25 and index[1] >= 25:
                        labelling[i] = ref[0] + ' ' + ref[1] + ' mixed group'
                    elif index[0] >= 25 and index[2] >= 25:
                        labelling[i] = ref[0] + ' ' + ref[2] + ' mixed group'
                    elif index[0] >= 25 and index[3] >= 25:
                        labelling[i] = ref[0] + ' ' + ref[3] + ' mixed group'
                    elif index[1] >= 25 and index[2] >= 25:
                        labelling[i] = ref[1] + ' ' + ref[2] + ' mixed group'
                    elif index[1] >= 25 and index[3] >= 25:
                        labelling[i] = ref[1] + ' ' + ref[3] + ' mixed group'
                    elif index[2] >= 25 and index[3] >= 25:
                        labelling[i] = ref[2] + ' ' + ref[3] + ' mixed group'
                else:
                    labelling[i] = 'Mixed investment group'
            return labelling
        
        def get_sharpe_ret(cumret_df, ret_df, cluster_label):
            if cumret_df.shape[1] != ret_df.shape[1] != len(cluster_label):
                raise ValueError('The input data are not consistent')

            sharpe_dict = {}
            absolute_return = {}
            for i in range(len(np.unique(cluster_label))):
                temp_dict = {}
                df = cumret_df[cumret_df.columns[cluster_label == i]]
                annual_return = (df.iloc[-1, :]) - 1
                annual_vol = ret_df[ret_df.columns[cluster_label == i]].std(axis=0)*np.sqrt(250)
                rf = 0.05 #assumption
                annual_sharpe_ratio = (annual_return - rf)/annual_vol
                pct_25 = np.round(annual_sharpe_ratio.describe()['25%'], 4)
                pct_75 = np.round(annual_sharpe_ratio.describe()['75%'], 4)
                low = np.round(pct_25 - (pct_75 - pct_25)*1.5, 4)
                high = np.round(pct_75 + (pct_75 - pct_25)*1.5, 4)
                for fund in annual_sharpe_ratio.index:
                    if annual_sharpe_ratio[fund] >= pct_25 and pct_75 > annual_sharpe_ratio[fund]:
                        sharpe_dict[fund] = f'medium: {pct_25}-{pct_75}'
                    elif annual_sharpe_ratio[fund] >= pct_75 and high > annual_sharpe_ratio[fund]:
                        sharpe_dict[fund] = f'high: {pct_75}-{high}'
                    elif annual_sharpe_ratio[fund] >= low and pct_25 > annual_sharpe_ratio[fund]:
                        sharpe_dict[fund] = f'low: {low}-{pct_25}'
                    elif annual_sharpe_ratio[fund] >= high:
                        sharpe_dict[fund] = f'very high: >{high}'
                    elif annual_sharpe_ratio[fund] < low:
                        sharpe_dict[fund] = f'very low: <{low}'

                temp_dict['med'] = list(annual_sharpe_ratio.index[(pct_75 > annual_sharpe_ratio) & (annual_sharpe_ratio >= pct_25)])
                temp_dict['high'] = list(annual_sharpe_ratio.index[(high > annual_sharpe_ratio) & (annual_sharpe_ratio >= pct_75)])
                temp_dict['very high'] = list(annual_sharpe_ratio.index[annual_sharpe_ratio >= high])
                temp_dict['low'] = list(annual_sharpe_ratio.index[(pct_25 > annual_sharpe_ratio) & (annual_sharpe_ratio >= low)])
                temp_dict['very low'] = list(annual_sharpe_ratio.index[annual_sharpe_ratio < low])
                for key in temp_dict.keys():
                    temp_df = cumret_df[temp_dict[key]]
                    final_return = (temp_df.iloc[-1, :]) - 1
                    threshold = final_return.median()
                    for j in final_return.index:
                        if final_return[j] >= threshold:
                            absolute_return[j] = ('high', np.round(final_return[j], 4))
                        else:
                            absolute_return[j] = ('low', np.round(final_return[j], 4))
            return sharpe_dict, absolute_return
        
        
        import os
        if not os.path.exists(loc):
            os.makedirs(loc)
        
        fund_ctgy = {self.mrnstar_data.iloc[j, :]['crsp_fundno']: self.mrnstar_data.iloc[j, :]['lipper_class_name'] for j in range(len(self.mrnstar_data))}
        cluster_label = label_cluster(self.label, self.feature_nostd)
        sharpe_rank, abret_rank = get_sharpe_ret(self.cumret_data, self.ret_data, self.label)
        df = pd.DataFrame(columns=['Fund.No', 'Ticker', 'Cluster'] + self.asset_type + ['Mstar Category', 
                                   'Cluster Category', 'sharpe_ratio', 'absolute_return', 'absolute_return_val'])
        df['Fund.No'] = np.array(self.feature_nostd.index)
        df['Ticker'] = df['Fund.No'].apply(lambda x: fundno_ticker[int(x)])
        df['Cluster'] = self.label
        for ass in self.asset_type:
            df[ass] = np.array(self.feature_nostd[ass])
        df['Mstar Category'] = df['Fund.No'].apply(lambda x: fund_ctgy.get(int(x), ''))
        df['Cluster Category'] = df['Cluster'].apply(lambda x: cluster_label[x])
        df['sharpe_ratio'] = df['Fund.No'].apply(lambda x: sharpe_rank[str(x)])
        df['absolute_return'] = df['Fund.No'].apply(lambda x: abret_rank[str(x)][0])
        df['absolute_return_val'] = df['Fund.No'].apply(lambda x: abret_rank[str(x)][1])
        df.to_csv(f'{loc}/cluster_result_{self.clustering_year}.csv', index=False)
        print('Sucessfully saved the clustering output!')

In [ ]:
# flc = first_layer_clustering(2019, data_trimmed, holding_asset, fund_mornstar)
# feature, feature_nostd = flc.init_features()
# label_2019 = flc.asset_based_clustering()
# flc.write_csv(fundno_ticker, loc='final_output')

In [ ]:
flc = first_layer_clustering(2019, data_trimmed, holding_asset, fund_mornstar)

In [ ]:
feature, feature_nostd = flc.init_features()

In [ ]:
label_2019 = flc.asset_based_clustering()

In [ ]:
flc.write_csv(fundno_ticker)

##### Stability check

In [ ]:
def stability(label1, label2, data1_nostd, data2_nostd, fund=False):
    #fund controls whether to show a specific fund's cluster change in two different time periods
    exceptions = []
    change_explainable, change_nonexplainable = [], []
    for i in range(len(label1)):
        fund = data1_nostd.index[i]
        if fund not in data2_nostd.index:
            exceptions.append(int(fund))
            continue

        no_1 = label1[i]
        no_2 = label2[list(data2_nostd.index).index(fund)]
        cluster_member_1 = set(data1_nostd.index[label1 == no_1].astype(int)) - {int(fund)}
        cluster_member_2 = set(data2_nostd.index[label2 == no_2].astype(int)) - {int(fund)}
        group1_index = data1_nostd[label1==no_1].median()
        group2_index = data2_nostd[label2==no_2].median()
        
        if len(cluster_member_1) == len(cluster_member_2) == 0:
            change_explainable.append(fund)
            continue
        if len(cluster_member_1) == 0:
            if all((data2_nostd.loc[fund, :] - group2_index) <= 10):
                change_explainable.append(fund)
                continue
            else:
                change_nonexplainable.append([fund])
                continue
        
        member_change_proportion = len(cluster_member_1.intersection(cluster_member_2))/len(cluster_member_1)
        if all((group1_index-group2_index).abs() <= 10) or member_change_proportion >= 0.7:
            continue
        if member_change_proportion < 0.7:
            holding_1 = data1_nostd.loc[fund, :]
            holding_2 = data2_nostd.loc[fund, :]
            change = (holding_1 - holding_2).abs()
            if all(change <= 10):
                if (sum(label1 == no_1) < 30 or sum(label2 == no_2) < 30):
                    change_explainable.append(fund)
                elif all(data1_nostd.loc[fund, :] - group1_index <= 5) and all(data2_nostd.loc[fund, :] - group2_index <= 5):
                    change_explainable.append(fund)
                else:
                    change_nonexplainable.append([fund, member_change_proportion])
            else:
                change_explainable.append(fund)
    
    if fund:
        print('The fund is in cluster', label2[list(data2_nostd.index).index(fund)], 'in previous year')
        print('The fund is in cluster', label1[list(data1_nostd.index).index(fund)], 'in later year')
        print()
        print('The asset info of the fund in previous year:')
        print(data2_nostd.loc[fund,:])
        print('The asset info of the fund in later year:')
        print(data1_nostd.loc[fund,:])
        print()
        print('The median asset info of the cluster where the fund is in previous year is:')
        print(data2_nostd[label2 == label2[list(data2_nostd.index).index(fund)]].median())
        print('The median asset info of the cluster where the fund is in later year is:')
        print(data1_nostd[label1 == label1[list(data1_nostd.index).index(fund)]].median())
        print()
    return change_explainable, change_nonexplainable, exceptions

In [ ]:
# change_explainable, change_nonexplainable, exceptions = stability(label_2019, label_2018, features_2019_nostd, features_2018_nostd)
# print(len(change_nonexplainable), len(change_explainable))

##### Second layer - subclustering based on result in first layer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tslearn
from tslearn.clustering import TimeSeriesKMeans
from sklearn import cluster
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances
from gap_statistic import OptimalK
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
import csv
import argparse
from time import time

# Keras
from keras.models import Model
from keras.layers import Input, Dense, Reshape, UpSampling2D, Conv2DTranspose, GlobalAveragePooling1D, Softmax
from keras.losses import kullback_leibler_divergence
import keras.backend as K

# scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans

# Dataset helper function
from ipynb.fs.full.datasets import load_data

# DTC components
from ipynb.fs.full.TSClusteringLayer import TSClusteringLayer
from ipynb.fs.full.TAE import temporal_autoencoder
from ipynb.fs.full.metrics import *
import ipynb.fs.full.tsdistances as tsdistances 

In [ ]:
class second_layer_clustering:
    
    def __init__(self, clustering_year, ret_data, asset_data, mrnstar_class_data, feature_first_layer, first_layer_result, group):
        self.clustering_year = clustering_year
        self.ret_data = ret_data[ret_data.index.year == clustering_year]
        self.cumret_data = (self.ret_data+1).cumprod()
        self.asset_data = asset_data[(asset_data.caldt.dt.year == clustering_year) & (asset_data.caldt.dt.month == 12)]
        self.mrnstar_data = mrnstar_class_data[(mrnstar_class_data.caldt.dt.year == (clustering_year)) & (mrnstar_class_data.caldt.dt.month == 12)]
        self.feature = feature_first_layer
        self.label = first_layer_result
        self.k = len(set(self.label))
        self.first_layer_output_csv = None
        if group >= self.k:
            raise ValueError('Group out of range')
        else:
            self.group = group
        
    def get_timeseries(self, ret=False, val=False):
        """
        ret: the daily return time series data of each fund
        val: the cumulative return time series data of each fund
        ret and val should at least provide one
        year: the clustering year, should be consistent with the first layer
        label: the clustering result from the first layer
        group: the specific group in the label to sub-cluster on
        feature: the feature data used in the first layer, to help match the fund number with the label
        """
        from sklearn import preprocessing
        if ret and val:
            ret = self.ret_data
            ret = ret[self.feature.index]
            ret = ret.iloc[:, self.label==self.group]
            val = (ret+1).cumprod()
            fundnos = ret.columns
            ret_train = ret.T
            val_train = val.T
            scaler1 = preprocessing.StandardScaler()
            scaler2 = preprocessing.StandardScaler()
            ret_train = scaler1.fit_transform(ret_train)
            val_train = scaler2.fit_transform(val_train)

            #reshape
            array1 = np.array(ret_train).reshape([ret_train.shape[0], ret_train.shape[1], -1])
            array2 = np.array(val_train).reshape([val_train.shape[0], val_train.shape[1], -1])
            res_train = np.concatenate((array1, array2), axis = 2)
            
            self.new_feature = res_train
            self.fundnos = fundnos
            return res_train, fundnos

        elif ret or val:
            df = self.ret_data
            df = df[self.feature.index]
            df = df.iloc[:, self.label==self.group]
            if ret:
                pass
            else:
                df = (df+1).cumprod()
            fundnos = df.columns
            df_train = df.T
            scaler1 = preprocessing.StandardScaler()
            df_train = scaler1.fit_transform(df_train)

            #reshape
            res_train = np.array(df_train).reshape([df_train.shape[0], df_train.shape[1], -1])
            
            self.new_feature = res_train
            self.fundnos = fundnos
            return res_train, fundnos

        else:
            raise ValueError('Should at least provide one')
            
    def organize_label(self, subcluster_label):
        label = subcluster_label
        label_set = set(label)
        number_of_cluster = len(label_set)
        for i in range(number_of_cluster):
            if i in label_set:
                continue
            else:
                mx = max(label_set)
                label = np.where(label==mx, i, label)
                label_set.add(i)
                label_set.remove(mx)
        self.subcluster_label = label
        self.subcluster_k = len(set(label))
        return label
    
    def plot_sub_result(self):
        k = self.subcluster_k
        width = 16
        height = ((k//10)+1)*15
        plt.figure(figsize=(width, height))
        for i in range(1, k+1):
            plt.subplot(int(np.ceil(k/4)), 4, i)
            for fund in self.fundnos[self.subcluster_label==(i-1)]:
                plt.plot(self.cumret_data[fund])
            plt.title(f'Cluster_{i-1} with {sum(self.subcluster_label==(i-1))} components')
        plt.show()

In [ ]:
#DTC Model -- first train the autoencoder to then train the clustering model
class DTC:
    """
    Deep Temporal Clustering (DTC) model
    # Arguments
        n_clusters: number of clusters
        input_dim: input dimensionality
        timesteps: length of input sequences (can be None for variable length)
        n_filters: number of filters in convolutional layer
        kernel_size: size of kernel in convolutional layer
        strides: strides in convolutional layer
        pool_size: pooling size in max pooling layer, must divide the time series length
        n_units: numbers of units in the two BiLSTM layers
        alpha: coefficient in Student's kernel
        dist_metric: distance metric between latent sequences
        cluster_init: cluster initialization method
    """

    def __init__(self, n_clusters, input_dim, timesteps,
                 n_filters=50, kernel_size=10, strides=1, pool_size=10, n_units=[50, 1],
                 alpha=1.0, dist_metric='eucl', cluster_init='kmeans'):
        assert(timesteps % pool_size == 0)
        self.n_clusters = n_clusters
        self.input_dim = input_dim
        self.timesteps = timesteps
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.pool_size = pool_size
        self.n_units = n_units
        self.latent_shape = (self.timesteps // self.pool_size, self.n_units[1])
        self.alpha = alpha
        self.dist_metric = dist_metric
        self.cluster_init = cluster_init
        self.pretrained = False
        self.model = self.autoencoder = self.encoder = self.decoder = None
    
    
    #Autoencoder part
    def initialize_autoencoder(self):
        """
        Create DTC model
        """
        # Create AE models
        self.autoencoder, self.encoder, self.decoder = temporal_autoencoder(input_dim=self.input_dim,
                                                                            timesteps=self.timesteps,
                                                                            n_filters=self.n_filters,
                                                                            kernel_size=self.kernel_size,
                                                                            strides=self.strides,
                                                                            pool_size=self.pool_size,
                                                                            n_units=self.n_units)
        
    def load_ae_weights(self, ae_weights_path):
        """
        Load pre-trained weights of AE
        # Arguments
            ae_weight_path: path to weights file (.h5)
        """
        self.autoencoder.load_weights(ae_weights_path)
        self.pretrained = True
        
    def pretrain(self, X,
                 optimizer='adam',
                 epochs=500,
                 batch_size=64,
                 save_dir='results',
                 verbose=1):
        """
        Pre-train the autoencoder using only MSE reconstruction loss
        Saves weights in h5 format.
        # Arguments
            X: training set
            optimizer: optimization algorithm
            epochs: number of pre-training epochs
            batch_size: training batch size
            save_dir: path to existing directory where weights will be saved
        """
        print('Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        # Begin pretraining
        t0 = time()
        self.autoencoder.fit(X, X, batch_size=batch_size, epochs=epochs, verbose=verbose)
        print('Pretraining time: ', time() - t0)
        self.autoencoder.save_weights('{}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        print('Pretrained weights are saved to {}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        self.pretrained = True
        
    def encode(self, x):
        """
        Encoding function. Extract latent features from hidden layer
        # Arguments
            x: data point
        # Return
            encoded (latent) data point
        """
        self.features = self.encoder.predict(x)
        return self.features

    def decode(self, x):
        """
        Decoding function. Decodes encoded sequence from latent space.
        # Arguments
            x: encoded (latent) data point
        # Return
            decoded data point
        """
        return self.decoder.predict(x)  
    
    #Clustering part
    def compile_clustering_model(self, optimizer):
        """
        Compile DTC model
        # Arguments
            gamma: coefficient of TS clustering loss
            optimizer: optimization algorithm
            initial_heatmap_loss_weight (optional): initial weight of heatmap loss vs clustering loss
            final_heatmap_loss_weight (optional): final weight of heatmap loss vs clustering loss (heatmap finetuning)
        """
        clustering_input = Input(shape=(self.features.shape[1], self.features.shape[2]), name='clustering_input')
        
        clustering_layer = TSClusteringLayer(self.n_clusters,
                                             alpha=self.alpha,
                                             dist_metric=self.dist_metric,
                                             name='TSClustering')(clustering_input)
        
        self.model = Model(inputs=clustering_input,
                           outputs=clustering_layer)
        print(self.model.summary())
        
        self.model.compile(loss='kld',
                           optimizer=optimizer)
    
    #Initialize cluster centers
    def init_cluster_weights(self):
        """
        Initialize with complete-linkage hierarchical clustering or k-means.
        # Arguments
            X: numpy array containing training set or batch
        """
        assert(self.cluster_init in ['hierarchical', 'kmeans'])
        print('Initializing cluster...')
        
        features = self.features

        if self.cluster_init == 'hierarchical':
            if self.dist_metric == 'eucl':  # use AgglomerativeClustering off-the-shelf
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='euclidean',
                                             linkage='complete').fit(features.reshape(features.shape[0], -1))
            else:  # compute distance matrix using dist
                d = np.zeros((features.shape[0], features.shape[0]))
                for i in range(features.shape[0]):
                    for j in range(i):
                        d[i, j] = d[j, i] = self.dist(features[i], features[j])
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='precomputed',
                                             linkage='complete').fit(d)
            # compute centroid
            cluster_centers = np.array([features[hc.labels_ == c].mean(axis=0) for c in range(self.n_clusters)])
        elif self.cluster_init == 'kmeans':
            # fit k-means on flattened features
            km = KMeans(n_clusters=self.n_clusters, n_init=10).fit(features.reshape(features.shape[0], -1))
            cluster_centers = km.cluster_centers_.reshape(self.n_clusters, features.shape[1], features.shape[2])

        self.model.get_layer(name='TSClustering').set_weights([cluster_centers])
        print('Done!')
    
    @property
    def cluster_centers_(self):
        """
        Returns cluster centers
        """
        return self.model.get_layer(name='TSClustering').get_weights()[0]

    @staticmethod
    def weighted_kld(loss_weight):
        """
        Custom KL-divergence loss with a variable weight parameter
        """
        def loss(y_true, y_pred):
            return loss_weight * kullback_leibler_divergence(y_true, y_pred)
        return loss

    def load_weights(self, weights_path):
        """
        Load pre-trained weights of DTC model
        # Arguments
            weight_path: path to weights file (.h5)
        """
        self.model.load_weights(weights_path)
        self.pretrained = True

    def dist(self, x1, x2):
        """
        Compute distance between two multivariate time series using chosen distance metric
        # Arguments
            x1: first input (np array)
            x2: second input (np array)
        # Return
            distance
        """
        if self.dist_metric == 'eucl':
            return tsdistances.eucl(x1, x2)
        elif self.dist_metric == 'cid':
            return tsdistances.cid(x1, x2)
        elif self.dist_metric == 'cor':
            return tsdistances.cor(x1, x2)
        elif self.dist_metric == 'acf':
            return tsdistances.acf(x1, x2)
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')

    def predict(self, x):
        """
        Predict cluster assignment.
        """
        q = self.model.predict(x, verbose=0)
        return q.argmax(axis=1)

    @staticmethod
    def target_distribution(q):  # target distribution p which enhances the discrimination of soft label q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def fit(self, X_train, y_train=None,
            X_val=None, y_val=None,
            epochs=100,
            eval_epochs=10,
            save_epochs=10,
            batch_size=64,
            tol=0.001,
            patience=5,
            save_dir='results_v2'):
        """
        Training procedure
        # Arguments
           X_train: training set
           y_train: (optional) training labels
           X_val: (optional) validation set
           y_val: (optional) validation labels
           epochs: number of training epochs
           eval_epochs: evaluate metrics on train/val set every eval_epochs epochs
           save_epochs: save model weights every save_epochs epochs
           batch_size: training batch size
           tol: tolerance for stopping criterion
           patience: patience for stopping criterion
           save_dir: path to existing directory where weights and logs are saved
        """
        if not self.pretrained:
            print('Autoencoder was not pre-trained!')

        # Logging file
        logfile = open(save_dir + '/dtc_log.csv', 'w')
        fieldnames = ['epoch', 'T', 'L', 'Lr', 'Lc']
        if X_val is not None:
            fieldnames += ['L_val', 'Lr_val', 'Lc_val']
        if y_train is not None:
            fieldnames += ['acc', 'pur', 'nmi', 'ari']
        if y_val is not None:
            fieldnames += ['acc_val', 'pur_val', 'nmi_val', 'ari_val']
        logwriter = csv.DictWriter(logfile, fieldnames)
        logwriter.writeheader()

        y_pred_last = None
        patience_cnt = 0

        print('Training for {} epochs.\nEvaluating every {} and saving model every {} epochs.'.format(epochs, eval_epochs, save_epochs))

        for epoch in range(epochs):

            # Compute cluster assignments for training set
            q = self.model.predict(X_train)
            p = DTC.target_distribution(q)

            # Evaluate losses and metrics on training set
            if epoch % eval_epochs == 0:

                # Initialize log dictionary
                logdict = dict(epoch=epoch)

                y_pred = q.argmax(axis=1)
                if X_val is not None:
                    q_val = self.model.predict(X_val)
                    p_val = DTC.target_distribution(q_val)
                    y_val_pred = q_val.argmax(axis=1)

                print('epoch {}'.format(epoch))
                loss = self.model.evaluate(X_train, p, batch_size=batch_size, verbose=False)
                logdict['L'] = loss
                print('[Train] - total loss={:f}'.format(logdict['L']))
                if X_val is not None:
                    val_loss = self.model.evaluate(X_val, p_val, batch_size=batch_size, verbose=False)
                    logdict['L_val'] = val_loss
                    print('[Val] - total loss={:f}'.format(logdict['L_val']))

                # Evaluate the clustering performance using labels
                if y_train is not None:
                    logdict['acc'] = cluster_acc(y_train, y_pred)
                    logdict['pur'] = cluster_purity(y_train, y_pred)
                    logdict['nmi'] = metrics.normalized_mutual_info_score(y_train, y_pred)
                    logdict['ari'] = metrics.adjusted_rand_score(y_train, y_pred)
                    print('[Train] - Acc={:f}, Pur={:f}, NMI={:f}, ARI={:f}'.format(logdict['acc'], logdict['pur'],
                                                                                    logdict['nmi'], logdict['ari']))
                if y_val is not None:
                    logdict['acc_val'] = cluster_acc(y_val, y_val_pred)
                    logdict['pur_val'] = cluster_purity(y_val, y_val_pred)
                    logdict['nmi_val'] = metrics.normalized_mutual_info_score(y_val, y_val_pred)
                    logdict['ari_val'] = metrics.adjusted_rand_score(y_val, y_val_pred)
                    print('[Val] - Acc={:f}, Pur={:f}, NMI={:f}, ARI={:f}'.format(logdict['acc_val'], logdict['pur_val'],
                                                                                  logdict['nmi_val'], logdict['ari_val']))

                logwriter.writerow(logdict)

                # check stop criterion
                if y_pred_last is not None:
                    assignment_changes = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = y_pred
                if epoch > 0 and assignment_changes < tol:
                    patience_cnt += 1
                    print('Assignment changes {} < {} tolerance threshold. Patience: {}/{}.'.format(assignment_changes, tol, patience_cnt, patience))
                    if patience_cnt >= patience:
                        print('Reached max patience. Stopping training.')
                        logfile.close()
                        break
                else:
                    patience_cnt = 0

            # Save intermediate model and plots
            if epoch % save_epochs == 0:
                self.model.save_weights(save_dir + '/DTC_model_' + str(epoch) + '.h5')
                print('Saved model to:', save_dir + '/DTC_model_' + str(epoch) + '.h5')

            # Train for one epoch
            self.model.fit(X_train, p, epochs=1, batch_size=batch_size, verbose=False)

        # Save the final model
        logfile.close()
        print('Saving model to:', save_dir + '/DTC_model_final.h5')
        self.model.save_weights(save_dir + '/DTC_model_final.h5')

In [ ]:
# slc = second_layer_clustering(clustering_year=2019, ret_data=data_trimmed, asset_data=holding_asset, 
#                               mrnstar_class_data=fund_mornstar, feature_first_layer=feature_nostd, 
#                               first_layer_result=label_2019, group=10)

In [ ]:
# compressed_data, fundnos = slc.get_timeseries(ret=True, val=True)

In [ ]:
# parser = argparse.ArgumentParser(description='train', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# # parser.add_argument('--dataset', default='CBF', help='UCR/UEA univariate or multivariate dataset')
# #parser.add_argument('--validation', default=False, type=bool, help='use train/validation split')
# parser.add_argument('--ae_weights', default=None, help='pre-trained autoencoder weights')
# parser.add_argument('--n_clusters', default=None, type=int, help='number of clusters')
# parser.add_argument('--n_filters', default=50, type=int, help='number of filters in convolutional layer')
# parser.add_argument('--kernel_size', default=10, type=int, help='size of kernel in convolutional layer')
# parser.add_argument('--strides', default=1, type=int, help='strides in convolutional layer')
# parser.add_argument('--pool_size', default=12, type=int, help='pooling size in max pooling layer') #Encoder output will be (21, 2)
# parser.add_argument('--n_units', nargs=2, default=[50, 1], type=int, help='numbers of units in the BiLSTM layers')
# parser.add_argument('--gamma', default=1.0, type=float, help='coefficient of clustering loss')
# parser.add_argument('--alpha', default=1.0, type=float, help='coefficient in Student\'s kernel')
# parser.add_argument('--dist_metric', default='eucl', type=str, choices=['eucl', 'cid', 'cor', 'acf'], help='distance metric between latent sequences')
# parser.add_argument('--cluster_init', default='hierarchical', type=str, choices=['kmeans', 'hierarchical'], help='cluster initialization method')
# parser.add_argument('--pretrain_epochs', default=500, type=int)
# parser.add_argument('--pretrain_optimizer', default='adam', type=str)
# parser.add_argument('--epochs', default=1000, type=int)
# parser.add_argument('--optimizer', default='adam', type=str)
# parser.add_argument('--eval_epochs', default=20, type=int)
# parser.add_argument('--save_epochs', default=50, type=int)
# parser.add_argument('--batch_size', default=64, type=int)
# parser.add_argument('--tol', default=0.001, type=float, help='tolerance for stopping criterion')
# parser.add_argument('--patience', default=5, type=int, help='patience for stopping criterion')
# parser.add_argument('--save_dir', default='result_secondlayer')
# args = parser.parse_args(args=[])
# print(args)

In [ ]:
# if not os.path.exists(args.save_dir):
#     os.makedirs(args.save_dir)

In [ ]:
# args.n_clusters = 15
# dtc = DTC(n_clusters=args.n_clusters,
#           input_dim=compressed_data.shape[-1],
#           timesteps=compressed_data.shape[1],
#           n_filters=args.n_filters,
#           kernel_size=args.kernel_size,
#           strides=args.strides,
#           pool_size=args.pool_size,
#           n_units=args.n_units,
#           alpha=args.alpha,
#           dist_metric=args.dist_metric,
#           cluster_init=args.cluster_init)

In [ ]:
# dtc.initialize_autoencoder()

In [ ]:
# if args.ae_weights is None and args.pretrain_epochs > 0:
#     dtc.pretrain(X=compressed_data, optimizer=args.pretrain_optimizer,
#                  epochs=args.pretrain_epochs, batch_size=args.batch_size,
#                  save_dir=args.save_dir)
# elif args.ae_weights is not None:
#     dtc.load_ae_weights(args.ae_weights)

In [ ]:
# secondlayer_features = dtc.encode(compressed_data)
# dtc.compile_clustering_model(optimizer=args.optimizer)
# dtc.init_cluster_weights()

In [ ]:
# t0 = time()
# dtc.fit(secondlayer_features, None, None, None, args.epochs, args.eval_epochs, args.save_epochs, args.batch_size,
#         args.tol, args.patience, args.save_dir)
# print('Training time: ', (time() - t0))

In [ ]:
# pred_p = dtc.model.predict(secondlayer_features)
# subcluster_label = pred_p.argmax(axis=1)

In [ ]:
# subcluster_label = slc.organize_label(subcluster_label)

In [ ]:
# slc.plot_sub_result()

In [ ]:
#determine the pool size
def isPrime(n):
    import math
    if n > 1:
        if n == 2:
            return True
        if n % 2 == 0:
            return False
        for x in range(3, int(math.sqrt(n) + 1), 2):
            if n % x == 0:
                return False
        return True
    return False

##### Code realization

<li>Hyper parameter

In [ ]:
parser = argparse.ArgumentParser(description='train', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument('--dataset', default='CBF', help='UCR/UEA univariate or multivariate dataset')
#parser.add_argument('--validation', default=False, type=bool, help='use train/validation split')
parser.add_argument('--year', default=2019, type=int)
parser.add_argument('--ae_weights', default=None, help='pre-trained autoencoder weights')
parser.add_argument('--n_clusters', default=None, type=int, help='number of clusters')
parser.add_argument('--n_filters', default=50, type=int, help='number of filters in convolutional layer')
parser.add_argument('--kernel_size', default=10, type=int, help='size of kernel in convolutional layer')
parser.add_argument('--strides', default=1, type=int, help='strides in convolutional layer')
parser.add_argument('--pool_size', default=12, type=int, help='pooling size in max pooling layer') #Encoder output will be (21, 2)
parser.add_argument('--n_units', nargs=2, default=[50, 1], type=int, help='numbers of units in the BiLSTM layers')
parser.add_argument('--gamma', default=1.0, type=float, help='coefficient of clustering loss')
parser.add_argument('--alpha', default=1.0, type=float, help='coefficient in Student\'s kernel')
parser.add_argument('--dist_metric', default='eucl', type=str, choices=['eucl', 'cid', 'cor', 'acf'], help='distance metric between latent sequences')
parser.add_argument('--cluster_init', default='hierarchical', type=str, choices=['kmeans', 'hierarchical'], help='cluster initialization method')
parser.add_argument('--pretrain_epochs', default=500, type=int)
parser.add_argument('--pretrain_optimizer', default='adam', type=str)
parser.add_argument('--epochs', default=1000, type=int)
parser.add_argument('--optimizer', default='adam', type=str)
parser.add_argument('--eval_epochs', default=20, type=int)
parser.add_argument('--save_epochs', default=50, type=int)
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--tol', default=0.001, type=float, help='tolerance for stopping criterion')
parser.add_argument('--patience', default=5, type=int, help='patience for stopping criterion')
parser.add_argument('--save_dir', default='result_secondlayer')
args = parser.parse_args(args=[])
print(args)

In [ ]:
args.year = 2010

<li>First layer

In [ ]:
flc = first_layer_clustering(clustering_year=args.year, ret_data=data_trimmed, asset_data=holding_asset, 
                             mrnstar_class_data=fund_mornstar)
feature, feature_nostd = flc.init_features()
firstlayer_label = flc.asset_based_clustering()
flc.write_csv(fundno_ticker, loc='result_firstlayer')

<li> Second layer

In [ ]:
#create saving path
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

subcluster_dict = dict()
for group in range(len(set(firstlayer_label))):
    slc = second_layer_clustering(clustering_year=args.year, ret_data=data_trimmed, asset_data=holding_asset, 
                                  mrnstar_class_data=fund_mornstar, feature_first_layer=feature_nostd, 
                                  first_layer_result=firstlayer_label, group=group)
    compressed_data, fundnos = slc.get_timeseries(ret=True, val=True)
    
    #Check if the sample is bigger than 1
    if compressed_data.shape[0] == 1:
        subcluster_dict[fundnos[0]] = 0
        continue
    
    
    #determine the pool size
    if isPrime(compressed_data.shape[1]):
        compressed_data = compressed_data[:, :compressed_data.shape[1]-1, :]
    for i in range(10, compressed_data.shape[1]//2):
        if compressed_data.shape[1]%i == 0:
            args.pool_size = i
            break
    
    #initialize model
    args.n_clusters = min(15, sum(firstlayer_label==group)//2)
    dtc = DTC(n_clusters=args.n_clusters,
              input_dim=compressed_data.shape[-1],
              timesteps=compressed_data.shape[1],
              n_filters=args.n_filters,
              kernel_size=args.kernel_size,
              strides=args.strides,
              pool_size=args.pool_size,
              n_units=args.n_units,
              alpha=args.alpha,
              dist_metric=args.dist_metric,
              cluster_init=args.cluster_init)

    #Train autoencoder
    dtc.initialize_autoencoder()
    if args.ae_weights is None and args.pretrain_epochs > 0:
        dtc.pretrain(X=compressed_data, optimizer=args.pretrain_optimizer,
                     epochs=args.pretrain_epochs, batch_size=args.batch_size,
                     save_dir=args.save_dir)
    elif args.ae_weights is not None:
        dtc.load_ae_weights(args.ae_weights)

    #Fetch the compressed data/result from the autoencoder
    secondlayer_features = dtc.encode(compressed_data)

    #Compile model
    dtc.compile_clustering_model(optimizer=args.optimizer)

    #Apply hierarchical clustering to select initial cluster centers used in KMeans
    dtc.init_cluster_weights()

    #Train clustering algorithm by using KL divergence as loss
    t0 = time()
    dtc.fit(secondlayer_features, None, None, None, args.epochs, args.eval_epochs, args.save_epochs, args.batch_size,
            args.tol, args.patience, args.save_dir)
    print('Training time: ', (time() - t0))

    #Get the clustering result
    pred_p = dtc.model.predict(secondlayer_features)
    subcluster_label = pred_p.argmax(axis=1)
    subcluster_label = slc.organize_label(subcluster_label)

    #Plot it out to check
    # slc.plot_sub_result()

    #Write into the dictionary
    for i in range(len(fundnos)):
        subcluster_dict[fundnos[i]] = subcluster_label[i]

In [ ]:
def export_result(first_layer_result_path, save_loc, subcluster_dict, first_layer_label, year):
        import os
        if not os.path.exists(save_loc):
            os.makedirs(save_loc)
        df = pd.read_csv(first_layer_result_path)
        if len(subcluster_dict) != len(first_layer_label):
            raise ValueError('The subclustering for each cluster is not finished.')
        df.insert(int(np.where(df.columns=='Cluster')[0][0]+1), 'Subcluster', np.ones(len(df)))
        df['Subcluster'] = df['Fund.No'].apply(lambda x: subcluster_dict[str(x)])
        df.to_csv(f'{save_loc}/cluster_result_withsub_{year}.csv', index=False)
        print('Successfully saved the csv file.')
        return df

In [ ]:
export_result(first_layer_result_path=f'result_firstlayer/cluster_result_{args.year}.csv', 
              save_loc='result_final', subcluster_dict=subcluster_dict, 
              first_layer_label=firstlayer_label, year=args.year)